In [1]:
# Fake news detection system
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string


In [2]:
data_fake = pd.read_csv("Fake.csv")
data_true = pd.read_csv("True.csv")


In [3]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
data_fake["Class"]=0
data_true["Class"]=1

In [6]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [7]:
print(data_fake.index)  # Display all indices in data_fake


RangeIndex(start=0, stop=23481, step=1)


In [8]:
data_merge= pd.concat([data_fake,data_true],axis=0) 

In [9]:
data_merge.head()

,title,text,subject,date,Class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [10]:
data_merge.tail()

,title,text,subject,date,Class
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [11]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'Class'], dtype='object')

In [12]:
data = data_merge.drop(['title','subject','date'], axis=1)


In [13]:
data.isnull().sum()

text     0
Class    0
dtype: int64

In [14]:
data=data.sample(frac=1)

In [15]:
data.head()

,text,Class
18575,"BUEA/BAMENDA, Cameroon (Reuters) - Soldiers sh...",1
4228,"Donald Trump is a sexual predator, and more an...",0
14267,ANKARA (Reuters) - Turkey has not been given c...,1
17323,THE OBAMA ADMINISTRATION IS PUSHING FOLKS OFF ...,0
592,Donald Trump seems to think that he can bully ...,0


In [16]:
data.reset_index(inplace=True)

In [17]:
data.head()

,index,text,Class
0,18575,"BUEA/BAMENDA, Cameroon (Reuters) - Soldiers sh...",1
1,4228,"Donald Trump is a sexual predator, and more an...",0
2,14267,ANKARA (Reuters) - Turkey has not been given c...,1
3,17323,THE OBAMA ADMINISTRATION IS PUSHING FOLKS OFF ...,0
4,592,Donald Trump seems to think that he can bully ...,0


In [18]:
data.drop(['index'],axis=1,inplace=True)

In [19]:
data.columns

Index(['text', 'Class'], dtype='object')

In [20]:
data.head()

,text,Class
0,"BUEA/BAMENDA, Cameroon (Reuters) - Soldiers sh...",1
1,"Donald Trump is a sexual predator, and more an...",0
2,ANKARA (Reuters) - Turkey has not been given c...,1
3,THE OBAMA ADMINISTRATION IS PUSHING FOLKS OFF ...,0
4,Donald Trump seems to think that he can bully ...,0


In [21]:
import re
import string

def wordrop(text):
    # Convert text to lowercase
    text = text.lower() 
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove digits
    text = re.sub(r'\d', '', text) 
    
    # Remove inline character
    text = re.sub(r'\n', '', text)
   
    return text


In [22]:
# Apply the 'wordrop' function to the 'text' column
data['text'] = data['text'].apply(wordrop)

In [23]:
data['text']

0        bueabamenda cameroon reuters  soldiers shot de...
1        donald trump is a sexual predator and more and...
2        ankara reuters  turkey has not been given cruc...
3        the obama administration is pushing folks off ...
4        donald trump seems to think that he can bully ...
                               ...                        
44893    st century wire saysupdate  at pm et due to pu...
44894    london reuters  british police said they carri...
44895    st century wire says legacy note to obama s gh...
44896    why do we even need a congress anymore we have...
44897    one of the countries mooch and her taxpayer fu...
Name: text, Length: 44898, dtype: object

In [24]:
x=data['text']
y=data['Class']

In [25]:
x

0        bueabamenda cameroon reuters  soldiers shot de...
1        donald trump is a sexual predator and more and...
2        ankara reuters  turkey has not been given cruc...
3        the obama administration is pushing folks off ...
4        donald trump seems to think that he can bully ...
                               ...                        
44893    st century wire saysupdate  at pm et due to pu...
44894    london reuters  british police said they carri...
44895    st century wire says legacy note to obama s gh...
44896    why do we even need a congress anymore we have...
44897    one of the countries mooch and her taxpayer fu...
Name: text, Length: 44898, dtype: object

In [26]:
y

0        1
1        0
2        1
3        0
4        0
        ..
44893    0
44894    1
44895    0
44896    0
44897    0
Name: Class, Length: 44898, dtype: int64

In [27]:
x_train,x_test,y_train,y_test,=train_test_split(x,y,test_size=0.3)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the vectorizer
vectorization = TfidfVectorizer()

# Fit and transform the training data, then transform the test data
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)


In [29]:
from sklearn.linear_model import LogisticRegression

# Initialize Logistic Regression model
LR = LogisticRegression()

# Train the model on the training data
LR.fit(xv_train, y_train)


LogisticRegression()

In [30]:
pred_lr=LR.predict(xv_test)

In [31]:
LR.score(xv_test,y_test)

0.9896065330363771

In [32]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7033
           1       0.99      0.99      0.99      6437

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [33]:
from sklearn.tree import DecisionTreeClassifier

DT=DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [34]:
pred_dt=DT.predict(xv_test)

In [35]:
DT.score(xv_test,y_test)

0.995025983667409

In [36]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7033
           1       0.99      0.99      0.99      6437

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [37]:
# Gradient boosting classifier
from sklearn.ensemble import GradientBoostingClassifier
GB=GradientBoostingClassifier(random_state=0)
GB.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [38]:
pred_gb=GB.predict(xv_test)

In [39]:
GB.score(xv_test,y_test)

0.9949517446176689

In [40]:
print(classification_report(y_test,pred_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      7033
           1       0.99      1.00      0.99      6437

    accuracy                           0.99     13470
   macro avg       0.99      1.00      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [41]:
from sklearn.ensemble import RandomForestClassifier

# Initialize Random Forest Classifier model
RF = RandomForestClassifier(random_state=0)

# Train the model on the training data
RF.fit(xv_train, y_train)


RandomForestClassifier(random_state=0)

In [42]:
pred_rf=RF.predict(xv_test)

In [43]:
RF.score(xv_test,y_test)

0.987305122494432

In [44]:
print(classification_report(y_test,pred_rf))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      7033
           1       0.98      0.99      0.99      6437

    accuracy                           0.99     13470
   macro avg       0.99      0.99      0.99     13470
weighted avg       0.99      0.99      0.99     13470



In [45]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Real News"
    else:
        return "Unknown"


In [46]:
def manual_testing(news):
    testing_news={"text": [news]}
    new_def_test=pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordrop)
    new_x_test=new_def_test["text"]
    new_xv_test=vectorization.transform(new_x_test)
    pred_LR=LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GB=GB.predict(new_xv_test)
    pred_RF=RF.predict(new_xv_test)
    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRF Prediction: {}".format(output_label(pred_LR[0]),
                                                                                                            output_label(pred_DT[0]),
                                                                                                            output_label(pred_GB[0]),
                                                                                                            output_label(pred_RF[0])))


In [52]:
# Take input from the user as an integer
news = str(input())  # Convert input to an integer

# Call the manual_testing function
manual_testing(news)


 On Friday, it was revealed that former Milwaukee Sheriff David Clarke, who was being considered for Homeland Security Secretary in Donald Trump s administration, has an email scandal of his own.In January, there was a brief run-in on a plane between Clarke and fellow passenger Dan Black, who he later had detained by the police for no reason whatsoever, except that maybe his feelings were hurt. Clarke messaged the police to stop Black after he deplaned, and now, a search warrant has been executed by the FBI to see the exchanges.Clarke is calling it fake news even though copies of the search warrant are on the Internet. I am UNINTIMIDATED by lib media attempts to smear and discredit me with their FAKE NEWS reports designed to silence me,  the former sheriff tweeted.  I will continue to poke them in the eye with a sharp stick and bitch slap these scum bags til they get it. I have been attacked by better people than them #MAGA I am UNINTIMIDATED by lib media attempts to smear and discredi



LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RF Prediction: Fake News


In [ ]:
news_artical=str(input())

In [49]:
manual_testing(news_artical)



LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RF Prediction: Fake News


In [50]:
raw_text = "The special counsel investigation of links between Russia and President Trump’s 2016 election campaign should continue without interference in 2018, despite calls from some Trump administration allies and Republican lawmakers to shut it down, a prominent Republican senator said on Sunday. Lindsey Graham, who serves on the Senate armed forces and judiciary committees, said Department of Justice Special Counsel Robert Mueller needs to carry on with his Russia investigation without political interference. “This investigation will go forward. It will be an investigation conducted without political influence,” Graham said on CBS’s Face the Nation news program. “And we all need to let Mr. Mueller do his job. I think he’s the right guy at the right time.” "

# Call the `manual_testing` function with the test news
clean_text = wordrop(raw_text)
print(clean_text)


the special counsel investigation of links between russia and president trumps  election campaign should continue without interference in  despite calls from some trump administration allies and republican lawmakers to shut it down a prominent republican senator said on sunday lindsey graham who serves on the senate armed forces and judiciary committees said department of justice special counsel robert mueller needs to carry on with his russia investigation without political interference this investigation will go forward it will be an investigation conducted without political influence graham said on cbss face the nation news program and we all need to let mr mueller do his job i think hes the right guy at the right time 


In [51]:
manual_testing(clean_text)



LR Prediction: Real News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RF Prediction: Real News
